In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('/kaggle/input/hamilton-lyrics/ham_lyrics.csv', encoding='ISO-8859-1')
df.head(5)

# Hamilton Lyrics - Named Entity Recognition

Named Entity Recognition (NER) is a branch of natural language processing (NLP) dedicated to extracting named entities from text. This notebook based on the hit musical Hamilton will attempt to extract the names of people from the script.

In [3]:
import re 

# get script as one single string
lines = [line for line in df['lines']]

### Building a training dataset
The training dataset that gets passed to SpaCy needs to contain sentence level extracts with the string indices of where the name begins and ends.

In [4]:
# find how names are used in the data 
# get list of sentences for training with some names in
names = ['Hamilton', 'Burr', 'Alexander', 'Eliza', 'George', 'Jefferson', 'Schyler', 'Thomas']
extracts = df[df['lines'].str.contains('|'.join(names))].reset_index()

# select only lines over 80 chars
training = [re.sub('[^A-z]', ' ', line) for line in df['lines'] if len(line)>30]

TRAIN = []

# for each line 
for line in training:
    # workout which name is included
    for name in names:
        found = re.search(name, line)
        if found:
            # strip out the start of the string
            starts = found.start()
            end = found.end()
            
            TRAIN.append((line, {"entities": [(starts, end, "PER")]}))
TRAIN[:2]

### Testing the pre-trained model
Before putting the training data to use a test is done on the pre-trained model:

In [7]:
import spacy
nlp = spacy.load('en_core_web_lg')
nlp.pipe_names

In [8]:
names_pretrained = []

# 430 names 
for line in lines:
    doc = nlp(line)
    for word in doc.ents:
        if word.label_ == 'PER':
            names_pretrained.append(word.text)

print(set(names_pretrained))
print(len(names_pretrained))

In [9]:
NER = nlp.get_pipe('ner')

# only change pipelines that need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

### Adding custom training and testing a trained model
Using the training samples above the new model is updated and tested below

In [10]:
import random
from spacy.util import minibatch, compounding
from spacy.training import Example
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(50):
    losses = {}
    for batch in spacy.util.minibatch(TRAIN, size=1):
        for text, annotations in batch:
            # create Example
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            # Update the model
            nlp.update([example], losses=losses, drop=0.3)

In [12]:
names = []

# 430 names 
for line in lines:
    doc = nlp(line)
    for word in doc.ents:
        if word.label_ == 'PER':
            names.append(word.text)
            
print(set(names))
print(len(names))

In [23]:
results = {}
for name in names:
    try:
        results[name] += 1
    except:
        results[name] = 1
        

### Plotting the results
To find how the pre-trained model did a bar chart is used

In [22]:
import plotly.express as px

results_df = pd.DataFrame({'Names': results.keys(), 'Count':results.values()}).sort_values('Count', ascending=False)
bar = px.bar(results_df, x='Names', y='Count', template='seaborn')
bar.show()